In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext nb_black

In [2]:
!pip install config

from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config
import pandas as pd
import csv

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

from utils.metrics import windowdiff, pk
from sklearn.metrics import precision_recall_fscore_support
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# import sys, os
# import config

# config.root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
# sys.path.insert(0, config.root_path)

In [4]:
def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)):
        if y_actual[i] == y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_actual[i] != y_pred[i]:
            FP += 1
        if y_actual[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_actual[i] != y_pred[i]:
            FN += 1
    return (TP, FP, TN, FN)


def get_precision(TP, FP, TN, FN):
    if TP == 0:
        return 0
    if TP + FP == 0:
        return 0
    return TP / (TP + FP)

def get_recall(TP, FP, TN, FN):
    if TP == 0:
        return 0
    if TP + FP == 0:
        return 0
    return TP / (TP + FN)

In [5]:
# complex
fiction_tuple = ("fiction", "57488", "BfCoW", "fiction") # dataset from, num samples, hash, dataset to
clinical_tuple = ("clinical", "38575", "En2mj", "clinical")
wiki_tuple = ("wiki", "37412", "6YPov", "wiki")
wiki_to_clinical_tuple = ("wiki", "37412", "6YPov", "clinical")

In [34]:
# hybrid conv - finetune, 10 epochs, complex
wiki_tuple = ("wiki", "40561", "iRZW8", "wiki")
fiction_tuple = ("fiction", "64277", "Bq8LT", "fiction")
clinical_tuple = ("clinical", "44979", "gZLlY", "clinical")

# conv - no finetune, 10 epochs, complex
wiki_tuple = ("wiki", "40561", "8Xjsz", "wiki")
fiction_tuple = ("fiction", "64277", "07DOI", "fiction")
clinical_tuple = ("clinical", "44979", "bFFOx", "clinical")

# simple (12/11/2022)
clinical_tuple = ("clinical", "24771", "eI2us", "clinical") # clinical, simple, no GSSD
wiki_tuple = ("wiki", "40561", "SwPSB", "wiki") # wiki, simple, no GSSD
fiction_tuple = ("fiction", "43721", "BzsWJ", "fiction") # wiki, simple, no GSSD

# simple finetuning (12/12/2022)
clinical_tuple = ("clinical", "24771", "du7Ee", "clinical") # clinical, simple, no GSSD, finetuning
wiki_tuple = ("wiki", "40561", "jA6HL", "wiki") # wiki, simple, no GSSD, finetuning
fiction_tuple = ("fiction", "DRh4S", "BzsWJ", "fiction") # wiki, simple, no GSSD, finetuning

In [71]:
model_type = "LDABERT2/simple"
lda_gamma = 15

model_config = clinical_tuple

model_signature = "{}-{}-1-pct-1-aug_{}/no-finetune".format(
    *model_config
)

predictions_directory = "{}/predictions/{}".format(config.root_path, model_config[2])
predictions_path = "{}/predictions.csv".format(predictions_directory)

# predictions_path = "{}/predictions/{}/predictions/{}/{}/predictions.csv".format(
#     config.root_path, model_hash, model_type, model_signature
# )

# # this is for when we want to test trained datasets and how they infer on another dataset.
# predictions_path = "{}/predictions/{}_{}-from-{}-to_lda-{}/predictions.csv".format(
#     config.root_path, model_config[2], model_config[0], model_config[3], lda_gamma
# )

# # new training log
# predictions_path = "{}/models/LDABERT2/hybrid-conv/{}-{}-1-pct-1-aug_{}/finetune-True/checkpoint/training.log".format(
#     config.root_path, model_config[0], model_config[1], model_config[2]
# )

In [72]:
predictions = []
reader = csv.DictReader(open(predictions_path, encoding="utf8"))
for row in reader:
    predictions.append(row)

In [73]:
dict(predictions[0])

{'prediction': '0',
 'ground_truth': '1',
 'text': 'Glaucoma  characterized by elevated intraocular pressure  optic disk cupping and atrophy  and loss of vision  is one of the three leading causes of acquired blindness in the Western world ',
 'lda_gamma': '15',
 'prediction_threshold': '0.5',
 'logit': '0.3854638934135437'}

In [74]:
from collections import OrderedDict

if type(predictions[0]) is OrderedDict:
  predictions = [dict(x) for x in predictions]

# predictions

In [79]:
prediction_thresholds = [
    #     0.25,
    #     0.3,
    #     0.35,
    #     0.4,
    #     0.45,
    #     0.475,
    #     0.49, # conv
    #     0.495, # conv
    #     0.499, # conv
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
    0.95,
    0.98,
    #     0.99,
    #     0.995,
    #     0.999,
    0.9999,
]

In [80]:
full_preds = []
for p in predictions:
    curr_preds = []
    for thresh in prediction_thresholds:
        curr_preds.append(1 if float(p["logit"]) > thresh else 0)
    full_preds.append([*curr_preds, p["ground_truth"]])
    print("pHat", *curr_preds, "| p", p["ground_truth"])

columns = [*prediction_thresholds, "ground_truth"]
data = full_preds

Streaming output truncated to the last 5000 lines.
pHat 1 0 0 0 0 0 0 0 | p 1
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 1 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 1 0 0 0 0 0 0 0 | p 0
pHat 1 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 1 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 0 0 0 0 0 0 0 0 | p 0
pHat 1 1 0 0 0 0 0 0 | p 0
pHat 1 0 0 0 0 0 0 0 | p 0
pHat

In [81]:
df = pd.DataFrame(data, columns=columns)
df

,0.5,0.6,0.7,0.8,0.9,0.95,0.98,0.9999,ground_truth
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6620,0,0,0,0,0,0,0,0,0
6621,0,0,0,0,0,0,0,0,0
6622,0,0,0,0,0,0,0,0,0
6623,0,0,0,0,0,0,0,0,0


In [82]:
ground_truth = [int(k) for k in df["ground_truth"].tolist()]
new_predictions = []

for p_t in prediction_thresholds:
    new_predictions.append(df[p_t].tolist())

print("TP, FP, TN, FN")
df_data = []
string_ground_truth = "".join([str(s) for s in ground_truth])

# evaluation_window = 10
evaluation_windows = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
]  # [2, 5, 7, 10, 14]
for evaluation_window in evaluation_windows:
    for thresh, pred_set in zip(prediction_thresholds, new_predictions):
        string_pred_set = "".join([str(s) for s in pred_set])

        pk_result = pk(string_pred_set, string_ground_truth, evaluation_window)
        window_diff_result = windowdiff(
            string_pred_set, string_ground_truth, evaluation_window
        )

        TP, FP, TN, FN = perf_measure(ground_truth, pred_set)
        precision = get_precision(TP, FP, TN, FN)
        recall = get_recall(TP, FP, TN, FN)

        # create the dataframe
        df_data.append(
            [
                evaluation_window,
                thresh,
                TP, FP, TN, FN,
                precision,
                recall,
                pk_result,
                window_diff_result,
            ]
        )

        print(thresh, (TP, FP, TN, FN), precision, recall)

TP, FP, TN, FN
0.5 (65, 1858, 4541, 161) 0.03380135205408216 0.28761061946902655
0.6 (17, 662, 5737, 209) 0.025036818851251842 0.0752212389380531
0.7 (1, 106, 6293, 225) 0.009345794392523364 0.004424778761061947
0.8 (0, 6, 6393, 226) 0 0
0.9 (0, 0, 6399, 226) 0 0
0.95 (0, 0, 6399, 226) 0 0
0.98 (0, 0, 6399, 226) 0 0
0.9999 (0, 0, 6399, 226) 0 0
0.5 (65, 1858, 4541, 161) 0.03380135205408216 0.28761061946902655
0.6 (17, 662, 5737, 209) 0.025036818851251842 0.0752212389380531
0.7 (1, 106, 6293, 225) 0.009345794392523364 0.004424778761061947
0.8 (0, 6, 6393, 226) 0 0
0.9 (0, 0, 6399, 226) 0 0
0.95 (0, 0, 6399, 226) 0 0
0.98 (0, 0, 6399, 226) 0 0
0.9999 (0, 0, 6399, 226) 0 0
0.5 (65, 1858, 4541, 161) 0.03380135205408216 0.28761061946902655
0.6 (17, 662, 5737, 209) 0.025036818851251842 0.0752212389380531
0.7 (1, 106, 6293, 225) 0.009345794392523364 0.004424778761061947
0.8 (0, 6, 6393, 226) 0 0
0.9 (0, 0, 6399, 226) 0 0
0.95 (0, 0, 6399, 226) 0 0
0.98 (0, 0, 6399, 226) 0 0
0.9999 (0, 0, 6399

In [87]:
df_evaluation_set = pd.DataFrame(
    df_data, columns=["K", "Threshold", "TP", "FP", "TN", "FN", "precision", "recall", "Pk", "WindowDiff"]
)

evaluation_file = "{}/eval.csv".format(predictions_directory)

try:
  # if not os.path.exists(os.path.dirname(evaluation_file)):
  #     os.makedirs(os.path.dirname(evaluation_file))
  with open(evaluation_file, 'w') as csvfile:
      df_evaluation_set.to_csv(evaluation_file, index=False)
except IOError:
    print("I/O error")

df_evaluation_set

,K,Threshold,TP,FP,TN,FN,precision,recall,Pk,WindowDiff
0,1,0.5000,65,1858,4541,161,0.033801,0.287611,0.304755,0.304755
1,1,0.6000,17,662,5737,209,0.025037,0.075221,0.131472,0.131472
2,1,0.7000,1,106,6293,225,0.009346,0.004425,0.049962,0.049962
3,1,0.8000,0,6,6393,226,0.000000,0.000000,0.035019,0.035019
4,1,0.9000,0,0,6399,226,0.000000,0.000000,0.034113,0.034113
...,...,...,...,...,...,...,...,...,...,...
115,15,0.8000,0,6,6393,226,0.000000,0.000000,0.408561,0.411133
116,15,0.9000,0,0,6399,226,0.000000,0.000000,0.407654,0.407654
117,15,0.9500,0,0,6399,226,0.000000,0.000000,0.407654,0.407654
118,15,0.9800,0,0,6399,226,0.000000,0.000000,0.407654,0.407654


In [88]:
print(df_evaluation_set.to_latex(index=False))

\begin{tabular}{rrrrrrrrrr}
\toprule
 K &  Threshold &  TP &   FP &   TN &  FN &  precision &   recall &       Pk &  WindowDiff \\
\midrule
 1 &     0.5000 &  65 & 1858 & 4541 & 161 &   0.033801 & 0.287611 & 0.304755 &    0.304755 \\
 1 &     0.6000 &  17 &  662 & 5737 & 209 &   0.025037 & 0.075221 & 0.131472 &    0.131472 \\
 1 &     0.7000 &   1 &  106 & 6293 & 225 &   0.009346 & 0.004425 & 0.049962 &    0.049962 \\
 1 &     0.8000 &   0 &    6 & 6393 & 226 &   0.000000 & 0.000000 & 0.035019 &    0.035019 \\
 1 &     0.9000 &   0 &    0 & 6399 & 226 &   0.000000 & 0.000000 & 0.034113 &    0.034113 \\
 1 &     0.9500 &   0 &    0 & 6399 & 226 &   0.000000 & 0.000000 & 0.034113 &    0.034113 \\
 1 &     0.9800 &   0 &    0 & 6399 & 226 &   0.000000 & 0.000000 & 0.034113 &    0.034113 \\
 1 &     0.9999 &   0 &    0 & 6399 & 226 &   0.000000 & 0.000000 & 0.034113 &    0.034113 \\
 2 &     0.5000 &  65 & 1858 & 4541 & 161 &   0.033801 & 0.287611 & 0.485054 &    0.491395 \\
 2 &     0.600

In [90]:
evaluation_type = "WindowDiff" # "WindowDiff"
for thresh_set in range(len(prediction_thresholds)):
    print()
    for ind in df_evaluation_set.index:
        # print only the ones needed for creating graphs
        if (df_evaluation_set["K"][ind] in [2,5,7,10,14]):
            if (ind % len(prediction_thresholds)) + 1 == thresh_set + 1:
                print(
                    (df_evaluation_set["K"][ind], df_evaluation_set[evaluation_type][ind])
                )


(2, 0.4913949275362319)
(5, 0.7627246639480441)
(7, 0.8454449312584983)
(10, 0.911577992744861)
(14, 0.9523593466424682)

(2, 0.23641304347826086)
(5, 0.4470623772843981)
(7, 0.5387520773530745)
(10, 0.6305925030229746)
(14, 0.7088626739261948)

(2, 0.09707125603864734)
(5, 0.21597945929617882)
(7, 0.28221785768242935)
(10, 0.36577992744860943)
(14, 0.45099818511796735)

(2, 0.06989734299516909)
(5, 0.1643256305694004)
(7, 0.22087928690134462)
(10, 0.2979141475211608)
(14, 0.3897459165154265)

(2, 0.06808574879227053)
(5, 0.16039873130946986)
(7, 0.2163468801933827)
(10, 0.2945888754534462)
(14, 0.38626739261947973)

(2, 0.06808574879227053)
(5, 0.16039873130946986)
(7, 0.2163468801933827)
(10, 0.2945888754534462)
(14, 0.38626739261947973)

(2, 0.06808574879227053)
(5, 0.16039873130946986)
(7, 0.2163468801933827)
(10, 0.2945888754534462)
(14, 0.38626739261947973)

(2, 0.06808574879227053)
(5, 0.16039873130946986)
(7, 0.2163468801933827)
(10, 0.2945888754534462)
(14, 0.3862673926194797